In [29]:
%matplotlib inline
import random
import torch
# import d2l
from d2l import torch as d2l

In [2]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))
    # print("X:",X)
    y = torch.matmul(X,w)+b
    # print("y:",y)
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1)) #将y变为n*1的矩阵

In [3]:
true_w = torch.tensor([2,-3.4])
# print(true_w.shape)
true_b = 4.2
features,labels = synthetic_data(true_w,true_b,1000)
# print(labels.shape)
# labels


In [4]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.6709,  0.7372]) 
label: tensor([0.3490])


In [41]:
#返回小批量样本
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = indices[i:min(i+batch_size, num_examples)]
        # batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)])
        yield features[batch_indices],labels[batch_indices]

In [42]:
batch_size = 10
for X,y in data_iter(batch_size,features,labels):
    print(X,'\n',y)
    break

tensor([[-0.2078, -0.7279],
        [-2.5238,  0.2009],
        [ 0.0928,  1.1192],
        [ 0.2884, -1.2504],
        [-1.9780, -0.4453],
        [ 0.9126, -0.0107],
        [ 1.3687,  0.2692],
        [-1.3664, -0.2164],
        [-0.4766,  0.2440],
        [-0.8831, -0.0706]]) 
 tensor([[ 6.2444],
        [-1.5320],
        [ 0.5819],
        [ 9.0119],
        [ 1.7529],
        [ 6.0533],
        [ 6.0153],
        [ 2.2116],
        [ 2.4248],
        [ 2.6636]])


In [39]:
#初始化模型参数
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [22]:
#定义模型
def linreg(X,w,b):
    return torch.matmul(X,w) + b

In [23]:
#定义损失函数
def squared_loss(y_hat,y):
    return (y_hat - y) ** 2 / 2

In [24]:
#优化算法 随机小批量梯度下降
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [40]:
#模拟三个迭代周期的情形
lr = 0.05
num_epochs = 5
net = linreg
loss = squared_loss
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w,b),y)  #计算损失函数，l.shape=(batch_size,1)
        l.sum().backward()
        sgd([w,b],lr,batch_size)   #更新参数
    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch{epoch+1},loss{float(train_l.mean()):f}')

epoch1,loss0.000773
epoch2,loss0.000051
epoch3,loss0.000051
epoch4,loss0.000052
epoch5,loss0.000051
